In [3]:
## import json
import os
import sys
from datetime import datetime

PRODUCTS_FILE = "products.json"
BILLS_FOLDER = "bills"
SALES_FILE = "sales.json"
cart = {}

# ------------------ Product Handling ------------------
def load_products():
    """Load products from file or create default ones"""
    if os.path.exists(PRODUCTS_FILE):
        with open(PRODUCTS_FILE, "r") as f:
            return json.load(f)
    else:
        return {
            "1": ["Milk", 120, 10],
            "2": ["Bread", 80, 20],
            "3": ["Rice", 250, 15],
            "4": ["Cold drink (04 pcs)", 800, 15],
            "5": ["Eggs (12 pcs)", 200, 12],
        }

def save_products(products):
    """Save products to file"""
    with open(PRODUCTS_FILE, "w") as f:
        json.dump(products, f)

# ------------------ Sales Handling ------------------
def load_sales():
    """Load total sales from file"""
    if os.path.exists(SALES_FILE):
        with open(SALES_FILE, "r") as f:
            return json.load(f)
    return {"total_sales": 0}

def save_sales(sales):
    """Save total sales to file"""
    with open(SALES_FILE, "w") as f:
        json.dump(sales, f)

# ------------------ Customer Functions ------------------
def show_products(products):
    print("\n--- Available Products ---")
    print("ID\tName\t\tPrice\tStock")
    for pid, details in products.items():
        print(f"{pid}\t{details[0]:15}\t{details[1]} Rs\t{details[2]}")
    print("--------------------------")

def add_to_cart(products):
    show_products(products)
    pid = input("Enter Product ID: ")
    if pid not in products:
        print("❌ Invalid Product ID")
        return
    qty = int(input("Enter Quantity: "))
    if qty > products[pid][2]:
        print("❌ Not enough stock!")
        return
    cart[pid] = cart.get(pid, 0) + qty
    print(f"✅ {products[pid][0]} added to cart.")

def view_cart(products):
    if not cart:
        print("🛒 Cart is empty.")
        return 0
    print("\n--- Your Cart ---")
    total = 0
    for pid, qty in cart.items():
        name, price, stock = products[pid]
        cost = qty * price
        total += cost
        print(f"{name} x {qty} = {cost} Rs")
    print("-------------------")
    print(f"Total = {total} Rs")
    return total

def generate_bill(products, total):
    """Generate bill and save as text file"""
    if not os.path.exists(BILLS_FOLDER):
        os.makedirs(BILLS_FOLDER)

    bill_number = len(os.listdir(BILLS_FOLDER)) + 1
    bill_filename = f"bill_{bill_number}.txt"
    bill_path = os.path.join(BILLS_FOLDER, bill_filename)

    with open(bill_path, "w") as f:
        f.write("===== Grocery Store Bill =====\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("-------------------------------\n")
        for pid, qty in cart.items():
            name, price, stock = products[pid]
            cost = qty * price
            f.write(f"{name} x {qty} = {cost} Rs\n")
        f.write("-------------------------------\n")
        f.write(f"Total Amount: {total} Rs\n")
        f.write("Thank you for shopping with us!\n")
        f.write("===============================\n")

    print(f"🧾 Bill generated: {bill_filename}")

def checkout(products):
    total = view_cart(products)
    if not cart:
        return
    confirm = input("Proceed to checkout? (y/n): ").lower()
    if confirm == "y":
        for pid, qty in cart.items():
            products[pid][2] -= qty
        save_products(products)

        # 📌 Update total sales
        sales = load_sales()
        sales["total_sales"] += total
        save_sales(sales)

        generate_bill(products, total)
        print("💵 Payment Successful! Thank you for shopping 🛒")
        cart.clear()

# ------------------ Admin Panel ------------------
def admin_panel(products):
    while True:
        print("\n--- Admin Panel ---")
        print("1. View Products")
        print("2. Add Product")
        print("3. Update Product")
        print("4. Remove Product")
        print("5. Back to Main Menu")
        print("6. View Total Sales")
        choice = input("Enter choice: ")

        if choice == "1":
            show_products(products)
        elif choice == "2":
            name = input("Product Name: ")
            price = int(input("Price: "))
            qty = int(input("Quantity: "))
            pid = str(max(map(int, products.keys())) + 1)
            products[pid] = [name, price, qty]
            save_products(products)
            print(f"✅ {name} added successfully.")
        elif choice == "3":
            show_products(products)
            pid = input("Enter Product ID to update: ")
            if pid in products:
                price = int(input("New Price: "))
                qty = int(input("New Quantity: "))
                products[pid][1] = price
                products[pid][2] = qty
                save_products(products)
                print("✅ Product updated successfully.")
            else:
                print("❌ Invalid Product ID")
        elif choice == "4":
            show_products(products)
            pid = input("Enter Product ID to remove: ")
            if pid in products:
                products.pop(pid)
                save_products(products)
                print("✅ Product removed.")
            else:
                print("❌ Invalid Product ID")
        elif choice == "6":
            sales = load_sales()
            print(f"💰 Total Sales: {sales['total_sales']} Rs")
        elif choice == "5":
            break
        else:
            print("❌ Invalid choice.")

# ------------------ Main Menu ------------------
def main_menu():
    products = load_products()
    while True:
        print("\n--- Grocery Store Management ---")
        print("1. Customer")
        print("2. Admin")
        print("3. Exit")
        choice = input("Enter choice: ")

        if choice == "1":
            while True:
                print("\n--- Customer Menu ---")
                print("1. View Products")
                print("2. Add to Cart")
                print("3. View Cart")
                print("4. Checkout")
                print("5. Back")
                ch = input("Enter choice: ")

                if ch == "1":
                    show_products(products)
                elif ch == "2":
                    add_to_cart(products)
                elif ch == "3":
                    view_cart(products)
                elif ch == "4":
                    checkout(products)
                elif ch == "5":
                    break
                else:
                    print("❌ Invalid choice.")

        elif choice == "2":
            password = input("Enter Admin Password: ")
            if password == "admin123":
                admin_panel(products)
            else:
                print("❌ Wrong Password")

        elif choice == "3":
            print("👋 Exiting... Thank you!")
            break
            #sys.exit()

        else:
            print("❌ Invalid choice.")

# ------------------ Run Program ------------------
if __name__ == "__main__":
    main_menu()


--- Grocery Store Management ---
1. Customer
2. Admin
3. Exit


Enter choice:  3


👋 Exiting... Thank you!
